In [207]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta

In [215]:
def nofluffjobs_page_job_offers() -> list:

    nofluffjobs_list = list()
    

    url = "https://nofluffjobs.com/pl/praca-zdalna/python?criteria=city%3Dwarszawa%20seniority%3Dtrainee,junior%20%20salary<pln12000m&page=1"

    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find("div", class_="list-container ng-star-inserted")
    
    for job_element in soup.select('a[class*="posting-list-item posting-list-item--"]'):
        
        
        nofluffjobs_dict = dict()
        company_element = job_element.find("span", class_="d-block posting-title__company text-truncate")
        job_title_element = job_element.find("h3", class_="posting-title__position text-truncate color-main ng-star-inserted")
        if "Fullstack" not in job_title_element.text and "DevOps" not in job_title_element.text:
            link_url = "http://nofluffjobs.com" + job_element["href"]
            page_job_element = requests.get(link_url)
            soup_page = BeautifulSoup(page_job_element.content, "html.parser")
            position = soup_page.find("span", class_="mr-10 font-weight-medium")
            publication_date_element = soup_page.find("div", class_="posting-time-row")
            days_after_publication = int(re.findall(r'\b\d+\b', publication_date_element.text.strip())[0])
            publication_date = (datetime.today() - (timedelta(days=days_after_publication))).strftime("%Y-%m-%d")
            #creation of job offers details dictionary
            
            nofluffjobs_dict["publication_date"] = publication_date
            nofluffjobs_dict["company"] = company_element.text.strip()
            nofluffjobs_dict["title"] = job_title_element.text.strip()
            nofluffjobs_dict["position"] = position.text.strip()
            nofluffjobs_dict["link_url"] = link_url

            nofluffjobs_list.append(nofluffjobs_dict)
    return nofluffjobs_list


In [220]:
import pandas as pd
import numpy as np

nofluffjobs_list = nofluffjobs_page_job_offers()
df_raw = pd.DataFrame.from_records(nofluffjobs_list)
df = df_raw.copy()
df['publication_date'] = pd.to_datetime(df['publication_date'], infer_datetime_format=True)
df.drop_duplicates(subset=['publication_date', 'company', 'title'], inplace=True, ignore_index=True)



In [221]:
df

,publication_date,company,title,position,link_url
0,2022-08-18,Amartus Polska Sp. z o.o.,Junior Phyton Developer,Junior,http://nofluffjobs.com/pl/job/junior-phyton-de...
1,2022-08-18,IDEA team within Narodowe Centrum Badań Jądrowych,Junior Python Developer,Junior,http://nofluffjobs.com/pl/job/junior-python-de...
2,2022-08-18,Devire,Python Developer (Data Science),Junior,http://nofluffjobs.com/pl/job/python-developer...
3,2022-08-20,Deloitte Advisory,AI LAB,Trainee,http://nofluffjobs.com/pl/job/ai-lab-deloitte-...
4,2022-08-19,Webinterpret,Junior Data Scientist,Junior,http://nofluffjobs.com/pl/job/junior-data-scie...
5,2022-08-08,Sport Vision Technology sp. z o.o. sp. k.,Junior AI Developer,Junior,http://nofluffjobs.com/pl/job/junior-ai-develo...
6,2022-08-17,Astrotectonic,Junior Data Scientist,Junior,http://nofluffjobs.com/pl/job/junior-data-scie...
